Code Contributors : Niko Pedraza, Adam Torek, Carson Keller
Link Tensorflow Documentation(version=2.17): https://www.tensorflow.org/api_docs/python/tf

In [4]:
import tensorflow as tf

from tensorflow.keras import layers, models,datasets

tensorflow.keras.application
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow.keras.applications.mobilenet_v2MobileNetV2'

Import Cifar100 dataset and normalize

In [3]:
#import cifar 100 which means it should have 10 classified/labels of image datasets
(train_images_cf100, train_labels_cf100), (test_images_cf100, test_labels_cf100) = datasets.cifar100.load_data()

(train_images, train_labels), (test_images, test_labels) = datasets.

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
